In [299]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.preprocessing import StandardScaler, Normalizer, scale
from sklearn.utils import resample
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, log_loss, auc, roc_curve, roc_auc_score, recall_score, precision_recall_curve
from sklearn.metrics import make_scorer, precision_score, fbeta_score, f1_score, classification_report

In [133]:
credit = pd.read_csv('https://query.data.world/s/3k2alo5tmp6n6jd75nwwgpvuf37sl4')

In [268]:
credit.columns = [x.lower() for x in credit.columns]
credit.rename(columns ={'fraud':'class'},inplace = True)

In [265]:
credit.head()

,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v22,v23,v24,v25,v26,v27,v28,amount,class,class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0,0


In [183]:
scaler = StandardScaler()
credit['amount'] = scaler.fit_transform(credit['amount'].values.reshape(-1, 1))

In [272]:
X = credit.iloc[:, :-1]
y = credit.iloc[:, -1]

In [273]:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size = .33, stratify = y, random_state = 0)

In [274]:
ratio = sum(ytrain == 0)/ sum(ytrain == 1)

In [275]:
# over sampling 1's

ytrain_pos_oversample = pd.concat([ytrain[ytrain==1]] * ratio, axis = 0)
Xtrain_pos_oversample = pd.concat([Xtrain.loc[ytrain==1, :]] * ratio, axis = 0)

In [277]:
# concat the repeated data with the original data together
final_ytrain = pd.concat([ytrain_pos_oversample,ytrain],axis =0).reset_index(drop = True)
final_Xtrain = pd.concat([Xtrain_pos_oversample,Xtrain],axis =0).reset_index(drop = True)

In [278]:
final_ytrain.value_counts(dropna = False, normalize = True)

1    0.500328
0    0.499672
Name: class, dtype: float64

In [283]:
# classifier with balanced dataset 
classifier = LogisticRegression()
Class_model = classifier.fit(final_Xtrain,final_ytrain)

In [296]:
pred_y = Class_model.predict(Xtest)
print 'Accuracy is :',accuracy_score(pred_y,ytest)
print 'Recall is:',recall_score(pred_y,ytest)
print 'precison is', precision_score(pred_y,ytest)
print 'roc_auc_score is:',roc_auc_score(pred_y,ytest)
print("\n confustion matrix on validation data: \n" + str(confusion_matrix(ytest, pred_y)))

Accuracy is : 0.983316841691
Recall is: 0.0835308056872
precison is 0.87037037037
roc_auc_score is: 0.541651642131

 confustion matrix on validation data: 
[[92278  1547]
 [   21   141]]
